In [260]:
import pandas as pd
from scipy import stats
import numpy as np

In [261]:
# data = pd.read_table('transactions.txt',sep=',',chunksize=10000, iterator=True)
data = pd.read_table('/usr/local/data/transactions.txt',sep=',',chunksize=10000, iterator=True)

for chunk in data:
    chunk.to_csv('data.csv', index = False)

df = pd.read_csv('data.csv',  header=None, names=["№", "id", "vol", "seg"])
df

,№,id,vol,seg
0,1,ADF346274,1000,R
1,2,JDY272800,600,AF
2,3,ADF346274,2460,R
3,4,TDG254776,4000,R
4,5,RYE846234,70000,AF


### Подсчитает количество клиентов для каждого из сегментов, совершавших транзакции. 

In [262]:
df['seg'].value_counts()

R     3
AF    2
Name: seg, dtype: int64

### Подсчитает средний объем отдельной транзакции в каждом из сегментов. 

In [263]:
df.groupby(['seg'])[['vol']].mean()

,vol
seg,
AF,35300.000000
R,2486.666667


### Подсчитает 90% доверительный интервал для среднего объема отдельной транзакции в каждом из сегментов. 

In [264]:
n = len(df.index)

In [265]:
# вместо 3 и 2 соответственно подставляется значений из таблицы в результате выполнения df['seg'].value_counts()
p = 3/n # 3 - вероятность исходов транзакций типа R
q = 2/n # 2 - вероятность исходов транзакций типа AF

In [266]:
z_value = stats.norm.ppf(q = 0.9)

# print(p - z_value * (np.sqrt((p * (1 - p)/n))), p + z_value * (np.sqrt((p * (1 - p)/n))))
print("Confidence interval for R: {0},{1}".format(p - z_value * (np.sqrt((p * (1 - p)/n))), 
                                            p + z_value * (np.sqrt((p * (1 - p)/n)))))

Confidence interval for R: 0.31922611957806474,0.8807738804219352


In [267]:
# print(q - z_value * (np.sqrt((q * (1 - q)/n))), q + z_value * (np.sqrt((q * (1 - q)/n))))
print("Confidence interval for AF: {0},{1}".format(q - z_value * (np.sqrt((q * (1 - q)/n))),
                                            q + z_value * (np.sqrt((q * (1 - q)/n)))))

Confidence interval for AF: 0.11922611957806478,0.6807738804219352


### Проверит гипотезу о равенстве средних объемов отдельных транзакций между сегментами при уровне значимости 10%. 

In [269]:
from scipy.stats import ttest_ind, ttest_rel

X = df[df['seg'] == 'R']['vol']
Y = df[df['seg'] == 'AF']['vol']

answer3 = ttest_ind(X, Y).pvalue
answer3

0.2944172804761428

Так как pvalue > 0.1, то можно подтвердить нулевую гипотезу о том, что средние объемы примерно равны для R и AF